In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.utils import SAVED_PICS_PATH, keras_dataset_image_preprocessing, DENSE_LAYER_NAME, CONV_LAYER_NAME, ALPHAS
from src.utils import SAVED_CLASSIFIER_PATH_FASHION, SAVED_CLADEC_PATH_FASHION, SAVED_REFAE_PATH_FASHION
from src.cladec import ClaDec, RefAE

(train_images, train_labels_c), (test_images, test_labels_c) = keras_dataset_image_preprocessing(
    keras.datasets.fashion_mnist)
SAVED_EVAL_PICS_PATH = SAVED_PICS_PATH.joinpath('fashion_mnist')

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_FASHION)
classifier.evaluate(test_images, test_labels_c)

2022-07-10 12:34:56.727394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:34:56.774687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:34:56.775038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 12:34:56.775604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

313/313 [==============================] - 2s 4ms/step - loss: 0.2974 - categorical_accuracy: 0.8910


[0.2973955273628235, 0.890999972820282]

In [4]:
decoder_dense = ClaDec.create_128_dense_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_FASHION.joinpath(f'{alpha:2}')
    cladec_dense = ClaDec.create_from_weights(cladec_path.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                              classifier, DENSE_LAYER_NAME, alpha / 100, decoder_dense)
    print("Evaluating ", cladec_path.joinpath(DENSE_LAYER_NAME))
    cladec_dense.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 0/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0283 - reconstruction_loss: 0.0285 - classification_loss: 1.5014
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 1/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0322 - reconstruction_loss: 0.0294 - classification_loss: 0.3265
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 5/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0478 - reconstruction_loss: 0.0328 - classification_loss: 0.3370
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/10/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0696 - reconstruction_loss: 0.0388 - classification_loss: 0.3470
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/25/my_dense
313/313 [==========================

In [5]:
decoder_conv = ClaDec.create_64_conv_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_FASHION.joinpath(f'{alpha:2}')
    cladec_conv = ClaDec.create_from_weights(cladec_path.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                             classifier, CONV_LAYER_NAME, alpha / 100, decoder_conv)
    print("Evaluating ", cladec_path.joinpath(CONV_LAYER_NAME))
    cladec_conv.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 0/my_conv
313/313 [==============================] - 2s 7ms/step - loss: 0.0043 - reconstruction_loss: 0.0043 - classification_loss: 0.4771
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 1/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0077 - reconstruction_loss: 0.0051 - classification_loss: 0.2693
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/ 5/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0211 - reconstruction_loss: 0.0081 - classification_loss: 0.2687
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/10/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0367 - reconstruction_loss: 0.0108 - classification_loss: 0.2705
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/cladec/25/my_conv
313/313 [==============================]

In [6]:
# load RefAE model
refae_dense = RefAE.create_from_weights(SAVED_REFAE_PATH_FASHION.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                        cladec_dense)
refae_conv = RefAE.create_from_weights(SAVED_REFAE_PATH_FASHION.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                       cladec_conv)
# get reference reconstructions
print("Evaluating ", SAVED_REFAE_PATH_FASHION.joinpath(DENSE_LAYER_NAME))
refae_dense.evaluate(test_images)
print("Evaluating ", SAVED_REFAE_PATH_FASHION.joinpath(CONV_LAYER_NAME))
refae_conv.evaluate(test_images)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/refae/my_dense
313/313 [==============================] - 1s 4ms/step - reconstruction_loss: 0.0085
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/fashion_mnist/refae/my_conv
313/313 [==============================] - 1s 4ms/step - reconstruction_loss: 0.0014


0.0014300914481282234